In [26]:
import open3d as o3d#open3d library for point cloud processing
import numpy as np
import subprocess#subprocess library to run c++ script inside python script
import os
import sys
import math
import matplotlib.pyplot as plt

In [2]:
def furthest_point_sampling(pcd,num_points):
    '''
    Furthest Point Sampling Algorithm
    Takes as input:
    pcd->point cloud(.ply) file
    num_points->user defined number of points for down-sampling
    Returns:new down-sampled point cloud(.ply)
    '''
    distances = pcd.compute_nearest_neighbor_distance()
    farthest_indices = np.zeros((num_points,), dtype=np.int32)
    seed = 0
    farthest_indices[0] = seed
    for i in range(1, num_points):
        seed = int(np.argmax(distances))
        farthest_indices[i] = seed
        distances[seed] = -1 
    downpcd=pcd.select_by_index(farthest_indices)
    cl, index = downpcd.remove_statistical_outlier(nb_neighbors=400,std_ratio=2)
    #outlier_cloud = downpcd.select_by_index(index, invert=True)
    return downpcd.select_by_index(index)
        

In [13]:
'''
...sample usage of Sample() class...
file_name: name of the point cloud file in .ply format with coordinates defined in x,y,z format
other point cloud file formats can also be used but the format specification needs to be changed in o3d.io.read_point_cloud function  

num_points=final number of down-sampled points (user defined)

Output:dpcd->down-sampled point cloud
o3d.visualization.draw_geometries:to visualize the dpcd point cloud

'''
file_name="cushion_1.ply"
num_points=500
pcd=o3d.io.read_point_cloud(file_name,format='xyz')
dpcd=furthest_point_sampling(pcd,num_points)
print([dpcd])
o3d.visualization.draw_geometries([dpcd])

[PointCloud with 490 points.]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [15]:
pcd=o3d.io.read_point_cloud('BR236-A236_1SR_DEF_6W-PP_SAE.ply',format='xyz')
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [5]:
def outlier_remove(pcd,nb_neighbours,std_ratio):
    cl, index = pcd.remove_statistical_outlier(nb_neighbours,std_ratio)
    inlier_cloud = pcd.select_by_index(index)
    outlier_cloud = pcd.select_by_index(index, invert=True)
    return pcd.select_by_index(index)

In [6]:
pcd=o3d.io.read_point_cloud("step_files/cushion_br297.ply")
dpcd=outlier_remove(pcd,5,5)
o3d.visualization.draw_geometries([dpcd])

In [45]:
def max_list(_list)->[float,float,float]:
    max_0,max_1,max_2=-2**20,-2**20,-2**20
    for ar in _list:
        max_0=max(max_0,ar[0])
        max_1=max(max_1,ar[1])
        max_2=max(max_2,ar[2])
    return max_0,max_1,max_2
def min_list(_list)->[float,float,float]:
    min_0,min_1,min_2=2**20,2**20,2**20
    for ar in _list:
        min_0=min(min_0,ar[0])
        min_1=min(min_1,ar[1])
        min_2=min(min_2,ar[2])
    return min_0,min_1,min_2
def avg_list(_list)->[float,float,float]:
    s_x,s_y,s_z=0,0,0
    size=len(_list)
    for ar in _list:
        s_x+=ar[0]
        s_y+=ar[1]
        s_z+=ar[2]
    if len(_list)==0:size=len(_list)+1
    return s_x/size,s_y/size,s_z/size

In [10]:
class Parametrize():
    '''
    cushion length
    cushion width
    insert width
    bolster width
    cushion angle
    bolster angle
    '''
    def __init__(self,pcd):
        self.points=np.asarray([pcd.points])
        if self.points.ndim==3:self.points=self.points[0]
        self.cushion_length=0
        self.cushion_width=0
        self.insert_width=0
        self.bolster_width=0
        self.cushion_angle=0
        self.bolster_angle=0
    def insert(self)->list:
        _max=np.max(self.points,axis=0)
        _min=np.min(self.points,axis=0)
        avg_y=float((_max[1]+_min[1])/2)
        avg_x=float((_max[0]+_min[0])/2)
        center_xy=[]
        for ar in self.points:
            if np.abs(float(ar[1])-avg_y)<1 and np.abs(float(ar[0])-avg_x)<1 :center_xy.append(ar)
        insert_center_z=max_list(center_xy)[2]
        list_cushion_angle=[]
        for ar in self.points:
            if np.abs(ar[2]-insert_center_z)<5:list_cushion_angle.append(ar)
        cushion_angle=np.abs(math.degrees(math.atan((list_cushion_angle[1][2]-list_cushion_angle[0][2])/(list_cushion_angle[1][0]-list_cushion_angle[0][0]))))
        list_insert_width=[]
        for ar in self.points:
            if np.abs(float(ar[2])-float(insert_center_z))<2 and np.abs(float(ar[0])-float(avg_x))<2:list_insert_width.append(ar)
        insert_width=np.abs(max_list(list_insert_width)[1]-min_list(list_insert_width)[1])
        list_cushion_length=[]
        for ar in self.points:
            if ar[2]==insert_center_z and np.abs(ar[1]-avg_y)<1:list_cushion_length.append(ar)
        list_insert_z_max_x_min=[]
        for ar in self.points:
            if ar[0]<avg_x and np.abs(ar[1]-avg_y)<1:list_insert_z_max_x_min.append(ar)
        insert_z_max_x_min=max_list(list_insert_z_max_x_min)[2]
        list_insert_x_min=[]
        list_insert_x_max=[]
        for ar in self.points:
            if ar[2]==insert_z_max_x_min:list_insert_x_min.append(ar)
        insert_x_min=min_list(list_insert_x_min)[0]
        for ar in self.points:
            if np.abs(ar[1]-avg_y)<1 and np.abs(ar[2]-insert_center_z)<5:list_insert_x_max.append(ar)
        insert_x_max=max_list(list_insert_x_max)[0]
        cushion_length=insert_x_max-insert_x_min
        insert_center_x=(insert_x_max+insert_x_min)/2
        list_cushion_center_z=[]
        for ar in self.points:
            if np.abs(ar[0]-insert_center_x)<1 and np.abs(ar[1]-avg_y)<1:list_cushion_center_z.append(ar)
        cushion_center_z=avg_list(list_cushion_center_z)[2]
        list_insert_center_line=[]
        for ar in self.points:
            if np.abs(ar[0]-insert_center_x)<1 :list_insert_center_line.append(ar)
        list_insert_center_line.sort(key=lambda x:x[1])
        bolster_max=max_list(list_insert_center_line)
        list_cushion_common=[]
        for i in range(int(len(list_insert_center_line)*0.3),int(len(list_insert_center_line)*0.7)):
            list_cushion_common.append(list_insert_center_line[i])
        cushion_common=max_list(list_cushion_common)
        bolster_width=np.abs(cushion_common[1]-bolster_max[1])
        cushion_width=2*bolster_width+insert_width
        bolster_angle=math.degrees(math.atan((cushion_common[2]-bolster_max[2])/(cushion_common[1]-bolster_max[1])))
        return insert_width,cushion_width,cushion_length,bolster_width,cushion_angle,bolster_angle

In [17]:
pcd=o3d.io.read_point_cloud("step_files/cushion_br297.ply")
dpcd=outlier_remove(pcd,5,5)
param=Parametrize(dpcd)
insert_width,cushion_width,cushion_length,bolster_width,cushion_angle,bolster_angle=param.insert()
print(f"insert width: {insert_width} \ncushion width: {cushion_width} \ncushion length: {cushion_length} \nbolster width: {bolster_width} \ncushion angle: {cushion_angle} \nbolster angle: {bolster_angle} \n")

insert width: 477.26399999999995 
cushion width: 748.2739999999999 
cushion length: 352.28 
bolster width: 135.505 
cushion angle: 19.210074095634802 
bolster angle: 31.075551336536055 



In [106]:
pcd=o3d.io.read_point_cloud("step_files/cushion_br297.ply")
dpcd=outlier_remove(pcd,5,5)
param=Parametrize(dpcd)
insert_width,cushion_width,cushion_length,bolster_width,cushion_angle,bolster_angle=param.insert()
print(f"insert width: {insert_width} \ncushion width: {cushion_width} \ncushion length: {cushion_length} \nbolster width: {bolster_width} \ncushion angle: {cushion_angle} \nbolster angle: {bolster_angle} \n")

insert width: 477.26399999999995 
cushion width: 748.2739999999999 
cushion length: 352.28 
bolster width: 135.505 
cushion angle: 19.210074095634802 
bolster angle: 31.075551336536055 



In [75]:
pcd=o3d.io.read_point_cloud("step_files/cushion_v530.ply")
dpcd=outlier_remove(pcd,5,5)
o3d.visualization.draw_geometries([dpcd])

In [107]:
pcd=o3d.io.read_point_cloud("step_files/cushion_v530.ply")
dpcd=outlier_remove(pcd,5,4)
param=Parametrize(dpcd)
insert_width,cushion_width,cushion_length,bolster_width,cushion_angle,bolster_angle=param.insert()
print(f"insert width: {insert_width} \ncushion width: {cushion_width} \ncushion length: {cushion_length} \nbolster width: {bolster_width} \ncushion angle: {cushion_angle} \nbolster angle: {bolster_angle} \n")

insert width: 342.75500000000005 
cushion width: 529.933 
cushion length: 300.68000000000006 
bolster width: 93.589 
cushion angle: 18.440475569264795 
bolster angle: 17.682740980171825 



In [80]:
def par(points,th=15):
    _max=np.max(points,axis=0)
    _min=np.min(points,axis=0)
    x_avg=float((_max[0]+_min[0])/2)
    y_avg=float((_max[1]+_min[1])/2)
    cl=[]
    for ar in points:
        if np.abs(ar[1]-y_avg)<1 :cl.append(ar)
    cl.sort(key=lambda x:x[0])
    _clh=[]
    zm=cl[0][2]
    pn=cl[0]
    for i in range(1,len(cl)):
        if int(cl[i-1][0])==int(cl[i][0]):
            zm=max(zm,cl[i][2])
            if zm==cl[i][2]:pn=cl[i]
        else:
            _clh.append(pn)
            zm=cl[i][2]
    clh=[]
    _clh.sort(key=lambda x:x[0])
    for i in range(len(_clh)-1):
        if _clh[i][0]!=_clh[i+1][0]:clh.append(_clh[i])
    clh.sort(key=lambda x:x[0])
    x_axis,z_axis=[],[]
    for i in range(len(clh)-1):
        x_axis.append(clh[i][0])
        z_axis.append(clh[i][2])
    new_z=[]
    new_x=[]
    new_z.append(z_axis[0])
    new_x.append(x_axis[0])
    for x,z in zip(x_axis[1:],z_axis[1:]):
        if np.abs(z-new_z[-1])<th:
            new_z.append(z)
            new_x.append(x)
    grad=[]
    for i in range(len(new_x)-1):
        grad.append((new_z[i+1]-new_z[i])/(new_x[i+1]-new_x[i]))
    lx=[]
    for i in range(len(new_x)-1):lx.append(new_x[i])
    dgrad=[]
    for i in range(1,len(lx)-1):
        dgrad.append((grad[i+1]-grad[i-1])/(2*(lx[i+1]-lx[i-1])))
    dfl_x=[]
    for d,x in zip(dgrad,lx):
        if np.abs(d)>0.1:dfl_x.append(x)
    x_cushion_min,x_cushion_max=0,0
    for i in range(len(dfl_x)-1):
        if dfl_x[i+1]-dfl_x[i]>50:
            x_cushion_min=dfl_x[i]
            x_cushion_max=dfl_x[i+1]
            break
    cushion_max=cl[0]
    cushion_min=cl[0]
    for ar in cl:
        if ar[0]==x_cushion_max:cushion_max=ar
        if ar[0]==x_cushion_min:cushion_min=ar
    cushion_angle=math.degrees(math.atan((cushion_max[2]-cushion_min[2])/(cushion_max[0]-cushion_min[0])))
    pl=[]
    for ar in points:
        if np.abs(ar[1]-y_avg)>40:pl.append(ar)
    z_bolster_max=max_list(pl)[2]
    l_z_bolster_max=[]
    for ar in points:
        if ar[2]==z_bolster_max and np.abs(ar[1]-y_avg)>20:l_z_bolster_max.append(ar)
    y_bolster_max=max_list(l_z_bolster_max)[1]
    y_bolster_min=min_list(l_z_bolster_max)[1]
    cushion_width=np.abs(y_bolster_max-y_bolster_min)
    pl=[]
    for ar in points:
        if np.abs(ar[1]-y_avg)>40:pl.append(ar)
    zb_max=max_list(pl)[2]
    l_zb_max=[]
    for ar in points:
        if ar[2]==zb_max and np.abs(ar[1]-y_avg)>40:l_zb_max.append(ar)
    max_b,min_b=max_list(l_zb_max),min_list(l_zb_max)
    bl=[]
    for ar in points:
        if ar[0]==max_b[0]:bl.append(ar)
    bl.sort(key=lambda x:x[1])
    zc=min_list(bl)[2]
    lc=[]
    for ar in bl:
        if np.abs(ar[2]-zc)<1:lc.append(ar)
    cl_max,cl_min=max_list(lc),min_list(lc)
    bolster_angle=math.degrees(math.atan((cl_max[2]-max_b[2])/(cl_max[1]-max_b[1])))
    bolster_width=2*np.abs(cl_max[1]-max_b[1])
    return cushion_max[0]-cushion_min[0],cushion_angle,cushion_width,bolster_angle,bolster_width

In [85]:
pcd=o3d.io.read_point_cloud("step_files/cushion_br297.ply")
dpcd=outlier_remove(pcd,5,4)
cushion_length,cushion_angle,cushion_width,bolster_angle,bolster_width=par(np.asarray([dpcd.points])[0])
print(f"cushion width: {cushion_width+bolster_width} \ncushion length: {cushion_length} \nbolster width: {bolster_width} \ncushion angle: {cushion_angle} \nbolster angle: {bolster_angle} \ninsert width:{cushion_width}")

cushion width: 581.518 
cushion length: 472.52 
bolster width: 178.87800000000004 
cushion angle: -14.684989673303313 
bolster angle: 32.14733565027893 
insert width:402.64000000000004


In [84]:
pcd=o3d.io.read_point_cloud("step_files/cushion_v530.ply")
dpcd=outlier_remove(pcd,5,4)
cushion_length,cushion_angle,cushion_width,bolster_angle,bolster_width=par(np.asarray([dpcd.points])[0],th=10)
print(f"cushion width: {cushion_width} \ncushion length: {cushion_length} \nbolster width: {bolster_width} \ncushion angle: {cushion_angle} \nbolster angle: {bolster_angle} \n")

C:\Users\VAVIKAS\AppData\Local\Temp\ipykernel_34524\2419913495.py:59: RuntimeWarning: invalid value encountered in scalar divide
  cushion_angle=math.degrees(math.atan((cushion_max[2]-cushion_min[2])/(cushion_max[0]-cushion_min[0])))


cushion width: 141.48199999999997 
cushion length: 0.0 
bolster width: 141.48199999999997 
cushion angle: nan 
bolster angle: 62.35681448143364 

